In [1]:
from toybox.interventions.amidar import *
from toybox.interventions.breakout import *
from toybox.interventions.base import *
from toybox.interventions.core import * 
from toybox import Toybox, Input

import numpy as np
import math
from matplotlib.pyplot import imshow, subplots, tight_layout
import random

Registered Toybox environments with gym.


In [2]:
def calc_distance(one, two):
    
    return abs(one.tx -  two.tx) + abs(one.ty - two.ty)